<a href="https://colab.research.google.com/github/HiteshAI/Plant_Classification/blob/master/nepali_sentiment_classifier_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.text import *
import pandas as pd

In [21]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving file_2.csv to file_2.csv
User uploaded file "file_2.csv" with length 27716 bytes


In [0]:
df = pd.read_csv('file_2.csv')

In [23]:
df.head()

,sentiments label
0,एमाले उपाध्यक्ष नारयणकाजी श्रेष्ठले बहुसदस्यीय...
1,तर त्यसमा गम्भीर छलफल नभएको अर्का एक नेताले बत...
2,एमाले महासचिव ईश्वर पोखरेलले संघीयतामा एमाओवाद...
3,धेरै प्रदेश बनाउने र जातीय पहिचान दिने एमाओवाद...
4,बैठकमा पोखरेल र संवैधानिक संवाद तथा राजनीतिक स...


In [15]:
df.keys()

Index(['sentiments\tlabel'], dtype='object')

In [0]:
path = Path('/content')
print(path)
data_lm = (TextList.from_csv(path, 'file_2.csv', cols='sentiments', delimiter='\t')
                   .split_by_rand_pct()
                   .label_for_lm()
                   .databunch())

In [32]:
data_lm.show_batch()

idx,text
0,xxunk xxunk xxunk र xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk थिए । xxbos xxunk उपाध्यक्ष xxunk xxunk भएको xxunk केही xxunk xxunk बताए । xxbos xxunk xxunk xxunk xxunk xxunk xxunk गरी छलफल xxunk भएको छ तर xxunk xxunk औपचारिक छलफल xxunk उनले भने । xxbos xxunk ८ xxunk xxunk xxunk xxunk xxunk xxunk थिए । xxbos तर एमाले अध्यक्ष xxunk xxunk xxunk
1,xxunk xxunk भएको xxunk केही xxunk xxunk बताए । xxbos xxunk xxunk xxunk xxunk xxunk xxunk गरी छलफल xxunk भएको छ तर xxunk xxunk औपचारिक छलफल xxunk उनले भने । xxbos xxunk ८ xxunk xxunk xxunk xxunk xxunk xxunk थिए । xxbos तर एमाले अध्यक्ष xxunk xxunk xxunk xxunk बैठकमा xxunk । xxbos xxunk xxunk xxunk औपचारिक xxunk xxunk xxunk xxunk भएको थियो । xxbos बैठकमा xxunk xxunk xxunk xxunk
2,xxunk औपचारिक छलफल xxunk उनले भने । xxbos xxunk ८ xxunk xxunk xxunk xxunk xxunk xxunk थिए । xxbos तर एमाले अध्यक्ष xxunk xxunk xxunk xxunk बैठकमा xxunk । xxbos xxunk xxunk xxunk औपचारिक xxunk xxunk xxunk xxunk भएको थियो । xxbos बैठकमा xxunk xxunk xxunk xxunk । xxbos xxunk xxunk कांग्रेस xxunk छलफल xxunk जानकारी xxunk । xxbos xxunk xxunk xxunk xxunk । xxbos xxunk xxunk xxunk xxunk xxunk
3,xxunk xxunk xxunk बैठकमा xxunk । xxbos xxunk xxunk xxunk औपचारिक xxunk xxunk xxunk xxunk भएको थियो । xxbos बैठकमा xxunk xxunk xxunk xxunk । xxbos xxunk xxunk कांग्रेस xxunk छलफल xxunk जानकारी xxunk । xxbos xxunk xxunk xxunk xxunk । xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk । xxbos xxunk xxunk xxunk उनी xxunk xxunk xxunk xxunk । xxbos xxunk xxunk xxunk xxunk xxunk xxunk लागि केही xxunk
4,xxunk । xxbos xxunk xxunk कांग्रेस xxunk छलफल xxunk जानकारी xxunk । xxbos xxunk xxunk xxunk xxunk । xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk । xxbos xxunk xxunk xxunk उनी xxunk xxunk xxunk xxunk । xxbos xxunk xxunk xxunk xxunk xxunk xxunk लागि केही xxunk xxunk xxunk जिल्ला प्रहरी xxunk xxunk थियो । xxbos xxunk xxunk xxunk xxunk लागि छ xxunk xxunk छन् । xxbos xxunk xxunk xxunk


In [33]:
learn = language_model_learner(data_lm, AWD_LSTM)
learn.fit_one_cycle(2, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,4.984819,3.838668,0.541071,00:11
1,5.004765,3.616324,0.547098,00:10


In [0]:
learn.save('mini_train_lm')
learn.save_encoder('mini_train_encoder')

In [35]:
learn.show_results()

text,target,pred
xxbos xxunk xxunk xxunk लाख xxunk xxunk छ । xxbos xxunk xxunk xxunk xxunk १ xxunk xxunk xxunk xxunk उनले,भने xxunk लागि एक xxunk xxunk xxunk xxunk xxunk xxunk xxunk छ । xxbos xxunk xxunk xxunk xxunk xxunk xxunk,xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxmaj xxunk xxunk xxunk xxunk
xxunk छ । xxbos xxunk xxunk xxunk xxunk १ xxunk xxunk xxunk xxunk उनले भने xxunk लागि एक xxunk xxunk,xxunk xxunk xxunk xxunk xxunk छ । xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk गर्न,xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk
xxunk xxunk १ xxunk xxunk xxunk xxunk उनले भने xxunk लागि एक xxunk xxunk xxunk xxunk xxunk xxunk xxunk छ,। xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk गर्न xxunk xxunk xxunk । xxbos xxunk,xxunk xxunk xxmaj xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj
xxunk उनले भने xxunk लागि एक xxunk xxunk xxunk xxunk xxunk xxunk xxunk छ । xxbos xxunk xxunk xxunk xxunk,xxunk xxunk xxunk xxunk xxunk xxunk xxunk गर्न xxunk xxunk xxunk । xxbos xxunk xxunk xxunk र xxunk xxunk xxunk,xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxmaj xxunk xxunk xxunk xxunk xxunk
xxunk xxunk xxunk xxunk xxunk xxunk xxunk छ । xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk,xxunk गर्न xxunk xxunk xxunk । xxbos xxunk xxunk xxunk र xxunk xxunk xxunk रहेको xxunk । xxbos xxunk xxunk,xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxmaj xxmaj


In [0]:
data_class = (TextList.from_csv(path, 'file_2.csv', cols='sentiments', delimiter='\t', vocab=data_lm.vocab)
                   .split_by_rand_pct(0.1)
                   .label_from_df(cols='label')
                   .databunch(bs=4))

In [43]:
data_class.show_batch()

text,target
xxbos xxunk दुर्घटना मा xxunk xxunk xxunk xxunk xxunk १ xxunk xxunk xxunk xxunk xxunk xxunk ८ वर्ष xxunk xxunk xxunk xxunk xxunk ८ वर्ष xxunk xxunk xxunk रसुवा जिल्ला भोर्ले xxunk को xxunk xxunk xxunk xxunk xxunk xxunk ४ वर्ष xxunk xxunk xxunk रसुवा जिल्ला भोर्ले ४ को xxunk xxunk xxunk xxunk xxunk xxunk ७ का वर्ष xxunk को xxunk xxunk रसुवा भोर्ले ४ का xxunk xxunk,-1
xxbos xxunk xxunk xxunk र xxunk xxunk xxunk xxunk xxunk xxunk xxunk भएको xxunk xxunk xxunk xxunk xxunk र xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk बताए ।,0
xxbos xxunk xxunk xxunk xxunk को xxunk xxunk xxunk xxunk गरी ३ करोड xxunk लाख xxunk xxunk xxunk xxunk भएको थियो भने ४ करोड xxunk xxunk xxunk गरिएको थियो ।,0
xxbos उपेन्द्रको xxunk xxunk xxunk xxunk xxunk छ त xxunk xxunk xxunk र xxunk xxunk छ xxunk xxunk xxunk xxunk उनले भने xxunk xxunk केही xxunk भएको छ ।,-1


In [44]:
learn = text_classifier_learner(data_clas, AWD_LSTM)
learn.load_encoder('mini_train_encoder')
learn.fit_one_cycle(2, slice(1e-3,1e-2))
learn.save('mini_train_clas')

epoch,train_loss,valid_loss,accuracy,time
0,1.251700,0.881208,0.615385,00:07
1,1.154357,0.904837,0.615385,00:06


In [45]:
learn.show_results()

text,target,prediction
xxbos औपचारिक बैठकमा एमाले अध्यक्ष xxunk xxunk xxunk र xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk थिए ।,0,0
xxbos दुर्घटना का घाइतेहरुको xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk र xxunk xxunk xxunk xxunk छ ।,-1,0
xxbos xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk छन् ।,0,1
xxbos xxunk xxunk xxunk देखि ७ लाख xxunk xxunk xxunk उनले बताए ।,0,1
